# Descarga de datos

## URL de las distintas fuentes de datos

**PRODUCCIÓN ANUAL AGRÍCOLA**
- https://nube.siap.gob.mx/cierreagricola/

**PRODUCCIÓN MENSUAL AGRÍCOLA**
- https://nube.siap.gob.mx/avance_agricola/
- http://infosiap.siap.gob.mx:8080/agricola_siap_gobmx/ResumenProducto.do

**INTENCION DE SIEMBRA**
- https://nube.siap.gob.mx/agroprograma/

**SISTEMA DE INFORMACIÓN AGROALIMENTARIA DE CONSULTA**
- http://infosiap.siap.gob.mx/gobmx/datosAbiertos_a.php

**ESTADÍSTICA DE USO TECNOLÓGICO Y DE SERVICIOS EN LA SUPERFICIE AGRÍCOLA**
- Fertilizada: https://nube.siap.gob.mx/index.php/s/kTLGLipCBEBiEFM/download
- Semilla mejorada y semilla criolla: https://nube.siap.gob.mx/index.php/s/OjKcZZL8QClaK6U/download
- Mecanizada: https://nube.siap.gob.mx/index.php/s/P7yK8NWmppStv7i/download
- Riego: https://nube.siap.gob.mx/index.php/s/cz6nETNW4b0krzD/download
- Sanidad: https://nube.siap.gob.mx/index.php/s/0l0Vw6VUf8fQ0dI/download
- Asistencia técnica: https://nube.siap.gob.mx/index.php/s/EHaUuYSEnHqHLqA/download

**INFORMACIÓN AGRÍCOLA DE GÉNERO**
- 2015 - 2017: https://www.gob.mx/cms/uploads/attachment/file/399835/Estadistica_de_G_nero_2017.pdf
- 2016 - 2018: https://www.gob.mx/cms/uploads/attachment/file/515428/Participaci_n_de_mujeres_en_valor_2016_17_18.pdf
- 2017 - 2019: https://www.gob.mx/cms/uploads/attachment/file/630415/Estadistica_de_Genero_2019.pdf

**MONOGRAFÍAS DE CULTIVOS**
- https://www.gob.mx/siap/documentos/monografias



## Librerias

In [ ]:
# !pip install lxml
# !pip install pandas

In [51]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import requests


## Catalogos para API: JSON de Victor

Nota: sí vienen los 64 cultivos que aparecen en la página

In [52]:
if not os.path.exists('catalogos_api'):
    os.mkdir("catalogos_api")

df = pd.read_json('Victor-dict.json',orient='index')

tag = ["municipio","anioagric","cicloProd","modalidad","entidad","cultivo","distrito","mesagric"]
name_csv = ['Municipio','Anio_Agricola','Ciclo_Prod','Modalidad','Entidad','Cultivo','Distrito',"Mes_Agricola"]

for i in range(8):
    # Toma los datos pertenecientes al tag
    df_aux2 = df.loc[df.index.str.startswith(tag[i])]
    df_aux = df_aux2.copy()
    # Elimina la str del tag+_ de la columna index
    df_aux.index = df_aux.index.str.replace(tag[i]+'_', "")
    # Convierte el indice en columna
    df_aux.reset_index(inplace=True)
    # Cambia los nombres de las columnas
    df_aux.rename(columns={0: "ID","index":"Name"}, inplace=True)
    # Crea el path donde se guardara
    ruta_completa = os.path.join("catalogos_api",name_csv[i])
    # Guardar el DataFrame en el archivo CSV en la carpeta "catalogos_api"
    df_aux.to_csv(ruta_completa + '.csv', index=False)

#Elimina los df
del df_aux, df_aux2

In [53]:
# df = pd.read_csv('./catalogos_api/anioagric.csv', index_col=False)


## Filtrar Municipios de Sonora

In [ ]:
# Define el enlace compartido del archivo en Google Drive
url = 'https://sistemadatos.sonora.gob.mx/OpenDataFiles/1573/2023050512492324-file.xlsx'

nombre_archivo = "catalogos.xlsx"  # Puedes ajustar el nombre según tu preferencia

# Realiza la solicitud GET para descargar el archivo
response = requests.get(url)

# Verifica si la descarga fue exitosa (código de respuesta 200)
if response.status_code == 200:
    # Abre un archivo local y guarda el contenido descargado
    with open(nombre_archivo, 'wb') as file:
        file.write(response.content)
    print(f"El archivo '{nombre_archivo}' se ha descargado correctamente.")
else:
    print("Error al descargar el archivo.")

El archivo 'catalogos.xlsx' se ha descargado correctamente.


In [ ]:
df = pd.read_excel('./catalogos.xlsx',sheet_name='Municipios')
df = df[['MUNICIPIOS','MUNI']]
df['Municipios'] = df['MUNICIPIOS'].str.lower()
df

,MUNICIPIOS,MUNI,Municipios
0,GENERAL PLUTARCO ELÍAS CALLES,70,general plutarco elías calles
1,ALTAR,4,altar
2,CANANEA,19,cananea
3,CABORCA,17,caborca
4,ATIL,7,atil
...,...,...,...
67,QUIRIEGO,49,quiriego
68,ETCHOJOA,26,etchojoa
69,NAVOJOA,42,navojoa
70,HUATABAMPO,33,huatabampo


In [ ]:
df2 = pd.read_csv('./catalogos_api/Municipio.csv')
df2['Name_min'] = df2['Name'].str.lower()
df2

,Name,ID,Name_min
0,Todos,0,todos
1,Aguascalientes,1,aguascalientes
2,Asientos,2,asientos
3,Calvillo,3,calvillo
4,Cosío,4,cosío
...,...,...,...
2322,Villa de Cos,51,villa de cos
2323,Villa García,52,villa garcía
2324,Villa González Ortega,53,villa gonzález ortega
2325,Villanueva,55,villanueva


In [ ]:
# Filtra las filas de df2 que coinciden con los municipios de df
df_filtrado = df2[df2['Name_min'].isin(df['Municipios'])]


In [ ]:
df_filtrado

,Name,ID,Name_min
167,Rayón,50,rayón
254,Rosario,51,rosario
268,Benito Juárez,4,benito juárez
573,Magdalena,98,magdalena
630,Villa Hidalgo,54,villa hidalgo
...,...,...,...
1878,Trincheras,64,trincheras
1879,Tubutama,65,tubutama
1880,Ures,66,ures
1881,Villa Pesqueira,68,villa pesqueira


In [ ]:
df2[df2['ID'] == 26]

,Name,ID,Name_min
107,Chenalhó,26,chenalhó
229,Gran Morelos,26,gran morelos
304,San Dimas,26,san dimas
338,Romita,26,romita
387,Cuetzala del Progreso,26,cuetzala del progreso
462,Huazalingo,26,huazalingo
545,Concepción de Buenos Aires,26,concepción de buenos aires
660,Chapa de Mota,26,chapa de mota
775,Chinicuila,26,chinicuila
888,Tlayacapan,26,tlayacapan


## Datos mensuales

In [55]:
# Creacion del folder raw
if not os.path.exists('raw'):
    os.mkdir("raw")

In [56]:
# Comando para ignorar warnings (salen en cada request)
warnings.filterwarnings('ignore')

In [57]:
anio_arr = pd.read_csv('./catalogos_api/Anio_Agricola.csv').ID.to_list()[:3]
cultivo_arr = pd.read_csv('./catalogos_api/Cultivo.csv').ID.to_list()
mes_arr = pd.read_csv('./catalogos_api/Mes_Agricola.csv').ID.to_list()

In [61]:
cultivo_arr.remove(0)

In [62]:
print(anio_arr)
print(cultivo_arr)
print(mes_arr)

[2023, 2022, 2021]
[6, 7, 8, 9, 15, 17, 27, 36, 42, 43, 53, 57, 60, 61, 63, 70, 74, 77, 78, 89, 105, 110, 112, 122, 128, 132, 149, 150, 152, 159, 163, 168, 175, 206, 212, 222, 224, 225, 230, 234, 241, 259, 268, 272, 283, 285, 289, 290, 296, 304, 320, 326, 373, 374, 375, 379, 389, 390, 392, 395, 400, 401, 416, 420]
[8, 7, 6, 5, 4, 3, 2, 1, 12, 11, 10, 9]


In [63]:
def API_SIAP(anio,mes,cultivo):
  # Endpoint
  url = "https://nube.siap.gob.mx/avance_agricola/"

  payload = {'xajax': 'reporte', # Para obtener la tabla
  'xajaxr': '1696449941927', # Timestamp UNIX
  'xajaxargs[]': [
      '1', # 1: Desglose por estados, 2: Cultivo total
      str(anio), # Anio
      '5', # ID Ciclo
      '3', # ID Modalidad
      '0', # ID Estado (0: Nacional)
      '--',
      '--',
      str(cultivo), # ID Cultivo
      '200201',
      '0',
      '1', # 1: Por municipio
      'undefined',
      'undefined',
      'undefined',
      str(mes) # Valor del mes: va de 1 a 8, con 1 siendo Enero y 8 Agosto
      ]
  }

  headers = {
    'Cookie': 'PHPSESSID=45ri2k73cbp2iptcrufu88p360'
  }

  # Hacemos el request
  response = requests.request("POST", url, headers=headers, data=payload, verify=False)
  response.encoding='ISO-8859-1'

  return response.text

In [76]:
# anio,mes,ciclo,cultivo, entidad, modalidad
columnas = ["Entidad", "Municipio", "Superficie Sembrada", "Superficie Cosechada", "Superficie Siniestrada", "Produccion", "Rendimiento","anio", "mes", "cultivo"]
df_raw = pd.DataFrame(columns=columnas)

for anio in anio_arr:
  for mes in mes_arr:
    for cultivo in cultivo_arr:
      xml_data = API_SIAP(anio,mes,cultivo)
      # Parseamos el XML con BeautifulSoup usando el analizador XML (lxml)
      soup = BeautifulSoup(xml_data, 'lxml');

      # Encontramos la tabla por su ID
      tabla = soup.find('table', id='Resultados-reporte')

      if tabla:
          # Inicializamos listas para almacenar los datos
          datos = []

          # Iteramos a través de las filas de la tabla
          for fila in tabla.find_all('tr')[1:]:  # Ignoramos la primera fila de encabezados
              columnas = fila.find_all('td')
              if len(columnas) == 0:  # Ignoramos las filas vacías
                  continue

              # Extraemos los datos relevantes de cada columna
              #posicion = columnas[0].get_text()
              entidad = columnas[1].get_text()
              municipio = columnas[2].get_text()
              superficie_sembrada = columnas[3].get_text()
              superficie_cosechada = columnas[4].get_text()
              superficie_siniestrada = columnas[5].get_text()
              produccion = columnas[6].get_text()
              rendimiento = columnas[7].get_text()

              # Agregamos los datos a la lista
              datos = [
                  entidad, municipio,
                  superficie_sembrada, superficie_cosechada,
                  superficie_siniestrada, produccion, rendimiento,
                  anio, mes, cultivo
              ]
              df_raw.loc[len(df_raw)] = datos

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
df_raw

,Entidad,Municipio,Superficie Sembrada,Superficie Cosechada,Superficie Siniestrada,Produccion,Rendimiento,anio,mes,cultivo
0,Aguascalientes,Aguascalientes,8.00,0.00,0.00,0.00,0,2023,8,6
1,Baja California Sur,La Paz,3.00,0.00,0.00,0.00,0,2023,8,6
2,Colima,Manzanillo,86.29,0.00,0.00,0.00,0,2023,8,6
3,Durango,Durango,13.00,0.00,0.00,0.00,0,2023,8,6
4,Durango,Mezquital,110.00,0.00,0.00,0.00,0,2023,8,6
...,...,...,...,...,...,...,...,...,...,...
61543,Zacatecas,Pánuco,1.00,0.00,0.00,0.00,0,2023,5,63
61544,Zacatecas,Pinos,13.00,0.00,0.00,0.00,0,2023,5,63
61545,Zacatecas,Tabasco,6.00,6.00,0.00,142.00,23.67,2023,5,63
61546,Zacatecas,Villa Hidalgo,77.76,0.00,0.00,0.00,0,2023,5,63


In [73]:
name_csv = "Avance_Agricola_2023_2022_2021"
ruta_completa = os.path.join("raw",name_csv)
df_raw.to_csv(ruta_completa + '.csv', index=False)

_____________________________

In [98]:
df_raw = pd.read_csv('./raw/Avance_Agricola_2023_2022_2021_2.csv')

In [99]:
df_raw

,Entidad,Municipio,Superficie Sembrada,Superficie Cosechada,Superficie Siniestrada,Produccion,Rendimiento,anio,mes,cultivo
0,Aguascalientes,Aguascalientes,8.00,0.00,0.00,0.00,0,2023,8,6
1,Baja California Sur,La Paz,3.00,0.00,0.00,0.00,0,2023,8,6
2,Colima,Manzanillo,86.29,0.00,0.00,0.00,0,2023,8,6
3,Durango,Durango,13.00,0.00,0.00,0.00,0,2023,8,6
4,Durango,Mezquital,110.00,0.00,0.00,0.00,0,2023,8,6
...,...,...,...,...,...,...,...,...,...,...
61543,Zacatecas,Pánuco,1.00,0.00,0.00,0.00,0,2023,5,63
61544,Zacatecas,Pinos,13.00,0.00,0.00,0.00,0,2023,5,63
61545,Zacatecas,Tabasco,6.00,6.00,0.00,142.00,23.67,2023,5,63
61546,Zacatecas,Villa Hidalgo,77.76,0.00,0.00,0.00,0,2023,5,63


In [100]:
catalogo_cultivos = pd.read_csv('./catalogos_api/Cultivo.csv')
catalogo_cultivos = catalogo_cultivos.set_index('ID')['Name'].to_dict()
df_raw['cultivo'] = df_raw['cultivo'].map(catalogo_cultivos)
df_raw

,Entidad,Municipio,Superficie Sembrada,Superficie Cosechada,Superficie Siniestrada,Produccion,Rendimiento,anio,mes,cultivo
0,Aguascalientes,Aguascalientes,8.00,0.00,0.00,0.00,0,2023,8,Agave
1,Baja California Sur,La Paz,3.00,0.00,0.00,0.00,0,2023,8,Agave
2,Colima,Manzanillo,86.29,0.00,0.00,0.00,0,2023,8,Agave
3,Durango,Durango,13.00,0.00,0.00,0.00,0,2023,8,Agave
4,Durango,Mezquital,110.00,0.00,0.00,0.00,0,2023,8,Agave
...,...,...,...,...,...,...,...,...,...,...
61543,Zacatecas,Pánuco,1.00,0.00,0.00,0.00,0,2023,5,Calabacita
61544,Zacatecas,Pinos,13.00,0.00,0.00,0.00,0,2023,5,Calabacita
61545,Zacatecas,Tabasco,6.00,6.00,0.00,142.00,23.67,2023,5,Calabacita
61546,Zacatecas,Villa Hidalgo,77.76,0.00,0.00,0.00,0,2023,5,Calabacita


In [101]:
catalogo_mes = pd.read_csv('./catalogos_api/Mes_Agricola.csv')
catalogo_mes = catalogo_mes.set_index('ID')['Name'].to_dict()
df_raw['mes'] = df_raw['mes'].map(catalogo_mes)

In [102]:
df_raw

,Entidad,Municipio,Superficie Sembrada,Superficie Cosechada,Superficie Siniestrada,Produccion,Rendimiento,anio,mes,cultivo
0,Aguascalientes,Aguascalientes,8.00,0.00,0.00,0.00,0,2023,Agosto,Agave
1,Baja California Sur,La Paz,3.00,0.00,0.00,0.00,0,2023,Agosto,Agave
2,Colima,Manzanillo,86.29,0.00,0.00,0.00,0,2023,Agosto,Agave
3,Durango,Durango,13.00,0.00,0.00,0.00,0,2023,Agosto,Agave
4,Durango,Mezquital,110.00,0.00,0.00,0.00,0,2023,Agosto,Agave
...,...,...,...,...,...,...,...,...,...,...
61543,Zacatecas,Pánuco,1.00,0.00,0.00,0.00,0,2023,Mayo,Calabacita
61544,Zacatecas,Pinos,13.00,0.00,0.00,0.00,0,2023,Mayo,Calabacita
61545,Zacatecas,Tabasco,6.00,6.00,0.00,142.00,23.67,2023,Mayo,Calabacita
61546,Zacatecas,Villa Hidalgo,77.76,0.00,0.00,0.00,0,2023,Mayo,Calabacita


In [105]:
df_raw.rename(columns={'anio':'Año','mes':'Mes','cultivo':'Cultivo'})

,Entidad,Municipio,Superficie Sembrada,Superficie Cosechada,Superficie Siniestrada,Produccion,Rendimiento,Año,Mes,Cultivo
0,Aguascalientes,Aguascalientes,8.00,0.00,0.00,0.00,0,2023,Agosto,Agave
1,Baja California Sur,La Paz,3.00,0.00,0.00,0.00,0,2023,Agosto,Agave
2,Colima,Manzanillo,86.29,0.00,0.00,0.00,0,2023,Agosto,Agave
3,Durango,Durango,13.00,0.00,0.00,0.00,0,2023,Agosto,Agave
4,Durango,Mezquital,110.00,0.00,0.00,0.00,0,2023,Agosto,Agave
...,...,...,...,...,...,...,...,...,...,...
61543,Zacatecas,Pánuco,1.00,0.00,0.00,0.00,0,2023,Mayo,Calabacita
61544,Zacatecas,Pinos,13.00,0.00,0.00,0.00,0,2023,Mayo,Calabacita
61545,Zacatecas,Tabasco,6.00,6.00,0.00,142.00,23.67,2023,Mayo,Calabacita
61546,Zacatecas,Villa Hidalgo,77.76,0.00,0.00,0.00,0,2023,Mayo,Calabacita
